<a href="https://colab.research.google.com/github/hojunking/carbon_reduction_project/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir_path = "/content/drive/MyDrive/탄소저감프로젝트/data"

img_names = []
img_labels = []
for folder, subfolders, filenames in os.walk(dir_path):
    for img in filenames:
        img_names.append(folder+'/'+img)
        img_labels.append(img)

print('Images: ',len(img_names))
print("Image_labels:", len(img_labels))

Images:  4067
Image_labels: 4067


In [ ]:
img_labels

In [ ]:
trn_df = pd.DataFrame(img_names, columns=['image_id'])
trn_df['dir'] = trn_df['image_id'].apply(lambda x: os.path.dirname(x))
trn_df['image_id'] = trn_df['image_id'].apply(lambda x: os.path.basename(x))
trn_df['label'] = img_labels
train = trn_df
train

,image_id,dir,label
0,건물 계단_422.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,건물 계단_422.jpg
1,건물 계단_424.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,건물 계단_424.jpg
2,건물 계단_425.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,건물 계단_425.jpg
3,건물 계단_426.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,건물 계단_426.jpg
4,건물 계단_427.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,건물 계단_427.jpg
...,...,...,...
4062,만보인증_222.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,만보인증_222.jpg
4063,만보인증_227.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,만보인증_227.jpg
4064,만보인증_224.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,만보인증_224.jpg
4065,만보인증_225.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,만보인증_225.jpg


In [ ]:
# Label Encoding
le = preprocessing.LabelEncoder()
train['artist'] = le.fit_transform(train['artist'].values)